- YOLO 모델로 inference 후 csv 파일을 확인해보니 예측이 아예 안 된 test img가 있다.
- 원래 csv 파일의 길이가 4871이 되어야 하는데, 개수가 모자른 것을 확인함
- 개수가 부족하면 앙상블이 안 되기 때문에 공백으로라도 채우려고 함

In [32]:
import pandas as pd

file_name = '/opt/ml/yolov5/fold_5_yolo.csv'

df=pd.read_csv(file_name)
print(len(df))

id_list = list(df['image_id'])
no_num_list = []


for i in range(len(df)-1):
    
    if int(id_list[i+1].split('/')[1].split('.')[0]) != int(id_list[i].split('/')[1].split('.')[0]) + 1:

        no_num_list.append(int(id_list[i].split('/')[1].split('.')[0])+1)


# 해당 번호를 아예 예측을 안해서 개수가 부족함
no_num_list

4871


[]

In [33]:
# 1자리, 2자리, 3자리 수면 앞에 0을 붙여야 됨

add_list = []

for i in no_num_list:
    add_zero = ''

    if len(str(i)) == 1:
        add_zero = '000' + str(i)

    elif len(str(i)) == 2:
        add_zero = '00' + str(i)

    elif len(str(i)) == 3:
        add_zero = '0' + str(i)

    else:
        add_zero = str(i)

    add_list.append(add_zero)

In [34]:
# 예측을 안 한 곳에 공백과 img 번호를 삽입
for i in add_list:
    new_data = {
        'PredictionString' : '',
        'image_id' : f'test/{i}.jpg'
    }
    
    idx = int(i) ## 원하는 인덱스
    
    temp1 = df[df.index < idx]
    temp2 = df[df.index >= idx]
    df = temp1.append(new_data,ignore_index=True).append(temp2, ignore_index=True)

In [35]:
len(df)

### 4871이 나오면 정상!!

4871

In [36]:
# 앙상블 할 폴더에 저장
df.to_csv(file_name, index=False)